In [1]:
from pptx import Presentation
from pptx.util import Inches, Pt
from glob import glob
from natsort import natsorted
import pandas as pd
import numpy as np
import math

from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.dml.color import RGBColor
import datetime
import pptx.util
#from pptx.util import Inches

In [2]:
# img_path = '/data/data1/hgamarro/data/NOAA_Hera_PhD_Work/JGR_2023/Cross_figs_0701_new'

# ox_list = natsorted(glob(img_path+'/Cross_Ox_**'))
# no2_list = natsorted(glob(img_path+'/Cross_no2**'))
# o3_list = natsorted(glob(img_path+'/Cross_o3**'))

# img_path_surf = '/data/data1/hgamarro/data/NOAA_Hera_PhD_Work/JGR_2023/Surf_figs_0701/'
# surf_ox_list = natsorted(glob(img_path_surf+'/Surf_**'))

# surf_ox_list = surf_ox_list[4:]

In [3]:
# img_path = '/data/data1/hgamarro/data/NOAA_Hera_PhD_Work/JGR_2023/Cross_figs_0701_new'

# ox_list = natsorted(glob(img_path+'/Cross_Ox_**'))
# no2_list = natsorted(glob(img_path+'/Cross_no2**'))
# o3_list = natsorted(glob(img_path+'/Cross_o3**'))

# # ox_list = ox_list[0:15]
# # no2_list = no2_list[0:15]
# # o3_list = o3_list[0:15]

# img_path_surf = '/data/data1/hgamarro/data/NOAA_Hera_PhD_Work/JGR_2023/Surf_figs_0701/'
# surf_ox_list = natsorted(glob(img_path_surf+'/Surf_**'))
# surf_ox_list = surf_ox_list[4:]

In [4]:
def extract_date_name_order(s):
    date_str, name = s.split('_')[1:][-2:]
    year, month, day = date_str.split('-')
    return (f'{year}-{month}-{day}', name_order[name])

In [5]:
### Timeseries Graphs ###

#df = pd.read_csv('/home/michael.bartsevich/SynopticPy/notebooks/2022_Obs_WRF/DATAFRAME_CSVs/2022-06-01_df.csv')
img_path = '/home/michael.bartsevich/SynopticPy/notebooks/2022_Obs_WRF/figures1'
pic_path = natsorted(glob(img_path+'/fig**'))
days_path = natsorted(glob(img_path+'/fig_????-??-??_CXO.png'))
str_list = pic_path

name_order = {'GLS.png': 1, 'HOU.png': 2, 'MCJ.png': 3, 'IAH.png': 4, 'DWH.png':5, 'CXO.png': 6, 'UTS.png': 7}
pic_path = sorted(pic_path, key=extract_date_name_order)
prs = Presentation()
prs.slide_width = Inches(16)
prs.slide_height = Inches(7)
for i in range(len(days_path)):
    date_text = days_path[i][72:82]

    blank_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_slide_layout)
    left = top = Inches(0)
    textbox = slide.shapes.add_textbox(left=Inches(12),
                                       top=Inches(6),
                                       width=Inches(3),
                                       height=Inches(0.5))
    textbox.text = date_text
    font = textbox.text_frame.paragraphs[0].font
    font.size = Pt(36)
    for j in zip(pic_path[7*i:7*(i+1)]):
        if left == 4*Inches(3.522):
            top = Inches(3.522)
            left = Inches(0)
        pic = slide.shapes.add_picture(j[0], left, top, 
                                       height=3200000,
                                       width=3200000
                                      )
        left = left + Inches(3.522)
        
prs.save('TimeseriesGraphs.pptx') 

In [6]:
def convert_to_float(x):
    try:
        return float(x)
    except ValueError:
        return x

In [7]:
def round_sigfigs(x, sigfigs=3):
    if isinstance(x, float) and not math.isnan(x):
        if x == 0:
            return 0
        else:
            return round(x, -int(math.floor(math.log10(abs(x))) - (sigfigs - 1)))
    else:
        return x

In [57]:
prs = Presentation()
prs.slide_width = Inches(16)
prs.slide_height = Inches(7)
#df = pd.read_csv('/home/michael.bartsevich/SynopticPy/notebooks/2022_Obs_WRF/ERROR_CSVs/2022-06-01_error.csv')
for s in errs_path:
    df = pd.read_csv(s)
    for i in df.columns:
        df[i] = df[i].apply(convert_to_float)
    # apply the function to each element in the dataframe using applymap()
    df = df.applymap(lambda x: round_sigfigs(x, sigfigs=3))
    df = df.drop(1)
    # specify the desired column order as a list
    column_order = ['Forecast','Unnamed: 1','HRRR.2', 'WRF.2', 'HRRR.3','WRF.3','HRRR','WRF','HRRR.1','WRF.1']
    # reorder the columns of the DataFrame based on the specified order
    df = df.reindex(columns=column_order)
    df = df.rename(columns={'Forecast': 'Station',
                            'Unnamed: 1': 'Variable',
                           'HRRR.2':'HRRR RMSE',
                           'HRRR.3':'HRRR MAE',
                           'HRRR':'HRRR R^2',
                           'HRRR.1':'HRRR MSE',
                           'WRF.2':'WRF RMSE',
                           'WRF.3':'WRF MAE',
                           'WRF':'WRF R^2',
                           'WRF.1':'WRF MSE'})
    df = df.drop(0)
    df = df.reset_index()
    df = df.drop(columns='index')
    df = df.set_index(['Station', 'Variable'])
    df.columns = pd.MultiIndex.from_tuples([('RMSE', 'HRRR'),('RMSE', 'WRF'),
                                            ('MAE', 'HRRR'), ('MAE', 'WRF'),
                                           ('R^2', 'HRRR'), ('R^2', 'WRF'),
                                           ('MSE', 'HRRR'), ('MSE', 'WRF'),])
    new_order = ['Temp', 'RH', 'WSpeed', 'WDir']
    df_reordered = df.reorder_levels(['Station', 'Variable'], axis=0)
    df = df_reordered.reindex(index=pd.MultiIndex.from_product([df_reordered.index.levels[0], new_order]))
    
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    
    textbox = slide.shapes.add_textbox(left=Inches(12),
                                       top=Inches(6),
                                       width=Inches(3),
                                       height=Inches(0.5))
    date_text = s[70:80]
    textbox.text = date_text
    font = textbox.text_frame.paragraphs[0].font
    font.size = Pt(36)

    left = Inches(0)
    top = Inches(0)
    width = Inches(8)
    height = Inches(4)

    table = slide.shapes.add_table(df.shape[0] + 2, df.shape[1]+2, left, top, width, height).table

    for i in [0,2,4,6]:
        cell = table.cell(0,i+2)
        cell.text = df.columns[i][0]

    for i in range(df.shape[1]):
        cell = table.cell(1, i+2)
        cell.text = df.columns[i][1]

    for i in [2,6,10,14,18,22,26]:
        cell = table.cell(i,0)
        cell.text = df.index[i][0]

    for i in range(df.shape[0]):
        cell = table.cell(i+2,1)
        cell.text = df.index[i][1]

    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            cell = table.cell(i+2, j+2)
            cell.text = str(df.iloc[i, j])

    for i in range(df.shape[0]):       
        for j in [2,4,6,8]:
            cell1 = table.cell(i+2,j)
            cell2 = table.cell(i+2,j+1)
            if j == 6:
                if float(cell1.text)>float(cell2.text):
                    fill = cell1.fill
                    fill.solid()
                    fill.fore_color.rgb = RGBColor(0, 176, 80)
                else:
                    fill = cell2.fill
                    fill.solid()
                    fill.fore_color.rgb = RGBColor(0, 176, 80)
            else:
                if float(cell1.text)>float(cell2.text):
                    fill = cell2.fill
                    fill.solid()
                    fill.fore_color.rgb = RGBColor(0, 176, 80)
                else:
                    fill = cell1.fill
                    fill.solid()
                    fill.fore_color.rgb = RGBColor(0, 176, 80)
    
    font_size = Pt(8)  # set the font size to 16 points
    for cell in table.iter_cells():
        cell.text_frame.paragraphs[0].font.size = font_size
    
    table.left = Inches(0)
    table.top = Inches(0)


prs.save('ErrorData.pptx')

In [207]:
# create a new PowerPoint presentation
prs = Presentation()

slide = prs.slides.add_slide(prs.slide_layouts[1])

left = Inches(1)
top = Inches(2)
width = Inches(8)
height = Inches(4)

table = slide.shapes.add_table(df.shape[0] + 2, df.shape[1]+2, left, top, width, height).table

for i in [0,2,4,6]:
    cell = table.cell(0,i+2)
    cell.text = df.columns[i][0]

for i in range(df.shape[1]):
    cell = table.cell(1, i+2)
    cell.text = df.columns[i][1]

for i in [2,6,10,14,18,22,26]:
    cell = table.cell(i,0)
    cell.text = df.index[i][0]

for i in range(df.shape[0]):
    cell = table.cell(i+2,1)
    cell.text = df.index[i][1]
    
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        cell = table.cell(i+2, j+2)
        cell.text = str(df.iloc[i, j])

for i in range(df.shape[0]):       
    for j in [2,4,6,8]:
        cell1 = table.cell(i+2,j)
        cell2 = table.cell(i+2,j+1)
        if j == 6:
            if float(cell1.text)>float(cell2.text):
                fill = cell1.fill
                fill.solid()
                fill.fore_color.rgb = RGBColor(0, 176, 80)
            else:
                fill = cell2.fill
                fill.solid()
                fill.fore_color.rgb = RGBColor(0, 176, 80)
        else:
            if float(cell1.text)>float(cell2.text):
                fill = cell2.fill
                fill.solid()
                fill.fore_color.rgb = RGBColor(0, 176, 80)
            else:
                fill = cell1.fill
                fill.solid()
                fill.fore_color.rgb = RGBColor(0, 176, 80)            


prs.save('example.pptx')
    

In [202]:
cell = table.cell(5,6)

In [205]:
float(cell.text)

0.632

In [206]:
df.shape[0]

28